# 02 - Latency & Throughput Benchmarking

## Learning Goals

* Distinguish **latency** (time for one request) vs **throughput** (requests/second).
* Understand **warm-up**: the first inferences are slower due to lazy initialization and caches.
* Interpret percentiles (**P50**, **P95**, **P99**) and why tail latency matters.
* See how **batch size** trades latency for throughput.

## You Should Be Able To...

- Explain why warm-up runs are necessary in latency benchmarking
- Run benchmarks with different batch sizes and interpret results
- Calculate and compare P50/P95 latency percentiles
- Identify performance bottlenecks in model inference
- Make informed decisions about batch size for deployment

---

## Concepts

**Warm-up runs**: prime kernels, JITs, memory. Don't include them in metrics.

**P50 vs P95**: P95 tells you about "slow outliers". SLAs often target a percentile, not the mean.

**Providers/EPs**: same ONNX model, different backends (CPU/GPU/NNAPI).

**Batch size**: larger batches can improve throughput but increase per-request latency.

## Success Criteria

* ✅ Report shows mean/P50/P95 and a PNG plot
* ✅ You can explain whether latency distribution is tight or spiky
* ✅ You can justify a batch size for your target use case

---

## Setup & Environment Check


In [ ]:
# ruff: noqa: E401
import os
import sys
from pathlib import Path

if Path.cwd().name == "labs":
    os.chdir(Path.cwd().parent)
    print("→ Working dir set to repo root:", os.getcwd())
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

import time
import numpy as np
import matplotlib.pyplot as plt
import onnxruntime as ort
from piedge_edukit.preprocess import FakeData
import piedge_edukit as _pkg  # noqa: F401


In [ ]:
# Environment self-heal (Python 3.12 + editable install)
import subprocess
import importlib

print(f"Python: {sys.version.split()[0]} (need 3.12)")

try:
    import piedge_edukit  # noqa: F401
    print("✅ PiEdge EduKit package OK")
except ModuleNotFoundError:
    print("ℹ️ Installing package in editable mode …")
    root = os.getcwd()
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", root])
    importlib.invalidate_caches()
    import piedge_edukit  # noqa: F401
    print("✅ Package installed")


In [ ]:
# All imports are now in the first cell above
print("✅ All imports successful")


## Concept: Latency vs Throughput

**Latency** measures how long a single inference takes (time per prediction).
**Throughput** measures how many inferences can be processed per second.

Key metrics:
- **Mean latency**: Average time per inference
- **P50 latency**: Median time (50% of inferences are faster)
- **P95 latency**: 95th percentile (95% of inferences are faster)
- **Warm-up**: Initial runs that "prime" the system (GPU memory allocation, JIT compilation, etc.)


## Task A: Explain Warm-up

**Multiple Choice**: Why are warm-up runs important in latency benchmarking?

A) They improve model accuracy
B) They initialize system resources (GPU memory, JIT compilation, etc.)
C) They reduce model size
D) They increase throughput

**Your answer**: _____

**Short justification** (1-2 sentences): Why does this matter for accurate benchmarking?

*Your answer here:*


## Task B: Batch Size Experiment

Run benchmarks with different batch sizes and analyze the performance trends.


In [ ]:
# TODO: run the benchmark with batch_size in {1, 8, 32}, runs=10
# Record p50/p95 and explain the trend.

def benchmark_batch_size(model_path, batch_sizes, runs=10, warmup=3):
    """
    Benchmark model with different batch sizes.
    Returns list of dicts with 'batch', 'p50', 'p95', 'mean' keys.
    """
    results = []
    
    # Load model once
    session = ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    
    for batch_size in batch_sizes:
        print(f"Benchmarking batch size {batch_size}...")
        
        # Generate test data
        fake_data = FakeData(num_samples=batch_size * runs, image_size=64, num_classes=2)
        latencies = []
        
        # Warm-up runs
        for _ in range(warmup):
            dummy_input = np.random.randn(batch_size, 3, 64, 64).astype(np.float32)
            _ = session.run([output_name], {input_name: dummy_input})
        
        # Actual benchmark runs
        for i in range(runs):
            dummy_input = np.random.randn(batch_size, 3, 64, 64).astype(np.float32)
            
            start_time = time.time()
            _ = session.run([output_name], {input_name: dummy_input})
            end_time = time.time()
            
            latency_ms = (end_time - start_time) * 1000
            latencies.append(latency_ms)
        
        # Calculate percentiles
        p50 = np.percentile(latencies, 50)
        p95 = np.percentile(latencies, 95)
        mean_lat = np.mean(latencies)
        
        results.append({
            'batch': batch_size,
            'p50': p50,
            'p95': p95,
            'mean': mean_lat
        })
        
        print(f"  Batch {batch_size}: P50={p50:.2f}ms, P95={p95:.2f}ms, Mean={mean_lat:.2f}ms")
    
    return results

# TODO: Run the experiment
# Hint: benchmark_batch_size("./models/student_model.onnx", [1, 8, 32], runs=10)

print("✅ Benchmark function ready")


In [ ]:
# Run the benchmark experiment
model_path = "./models/student_model.onnx"
if not os.path.exists(model_path):
    print("❌ Model not found. Please complete Notebook 01 first.")
    print("Expected path:", model_path)
else:
    # TODO: Run the benchmark with batch sizes [1, 8, 32]
    results = benchmark_batch_size(model_path, [1, 8, 32], runs=10)
    
    # Display results in a table
    print("\n📊 Benchmark Results:")
    print("Batch Size | P50 (ms) | P95 (ms) | Mean (ms)")
    print("-" * 45)
    for r in results:
        print(f"{r['batch']:10} | {r['p50']:8.2f} | {r['p95']:8.2f} | {r['mean']:8.2f}")
    
    # Auto-check
    assert len(results) >= 3 and all({'batch','p50','p95'} <= set(r) for r in results)
    print("✅ Results format OK")


In [ ]:
# Visualize the results
if 'results' in locals() and len(results) >= 3:
    batch_sizes = [r['batch'] for r in results]
    p50_values = [r['p50'] for r in results]
    p95_values = [r['p95'] for r in results]
    mean_values = [r['mean'] for r in results]
    
    plt.figure(figsize=(10, 6))
    plt.plot(batch_sizes, p50_values, 'o-', label='P50 Latency', linewidth=2)
    plt.plot(batch_sizes, p95_values, 's-', label='P95 Latency', linewidth=2)
    plt.plot(batch_sizes, mean_values, '^-', label='Mean Latency', linewidth=2)
    
    plt.xlabel('Batch Size')
    plt.ylabel('Latency (ms)')
    plt.title('Latency vs Batch Size')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print("📈 Chart shows latency trends across batch sizes")
else:
    print("⚠️ No results to visualize. Run the benchmark first.")


## Analysis Questions

Based on your benchmark results, answer these questions:

**1. How does latency change as batch size increases? Explain the trend.**

*Your answer here (2-3 sentences):*

---

**2. Why might P95 latency be higher than P50 latency? What does this tell us about system performance?**

*Your answer here (2-3 sentences):*

---

**3. If you were deploying this model to a real-time application, which batch size would you choose and why?**

*Your answer here (2-3 sentences):*


## Next Steps

Excellent work! You've learned how to measure and analyze model performance.

**Next**: Open `03_quantization.ipynb` to learn about model compression and optimization.

---

### Summary
- ✅ Understood latency vs throughput concepts
- ✅ Implemented warm-up benchmarking
- ✅ Analyzed batch size effects on performance
- ✅ Interpreted P50/P95 latency metrics


# ⚡ Latensbenchmark - Förstå modellens prestanda

**Mål**: Förstå hur vi mäter och tolkar modellens latens (svarstid).

I detta notebook kommer vi att:
- Förstå vad latens är och varför det är viktigt
- Se hur benchmark fungerar (warmup, runs, providers)
- Tolka resultat (p50, p95, histogram)
- Experimentera med olika inställningar

> **💡 Tips**: Latens är avgörande för edge deployment - en modell som är för långsam är inte användbar i verkligheten!


## 🤔 Vad är latens och varför är det viktigt?

**Latens** = tiden det tar för modellen att göra en förutsägelse (inference time).

**Varför viktigt för edge**:
- **Realtidsapplikationer** - robotar, autonoma fordon
- **Användarupplevelse** - ingen vill vänta 5 sekunder på en bildklassificering
- **Resursbegränsningar** - Raspberry Pi har begränsad CPU/memory

<details>
<summary>🔍 Klicka för att se typiska latensmål</summary>

**Typiska latensmål**:
- **< 10ms**: Realtidsvideo, gaming
- **< 100ms**: Interaktiva applikationer
- **< 1000ms**: Batch processing, offline analys

**Vår modell**: Förväntar oss ~1-10ms på CPU (bra för edge!)

</details>


## 🔧 Hur fungerar benchmark?

**Benchmark-processen**:
1. **Warmup** - kör modellen några gånger för att "värmma upp" (JIT compilation, cache)
2. **Runs** - mäter latens för många körningar
3. **Statistik** - beräknar p50, p95, mean, std

**Varför warmup?**
- Första körningen är ofta långsam (JIT compilation)
- Cache-värme påverkar prestanda
- Vi vill mäta "steady state" prestanda


In [ ]:
# Kör benchmark med olika inställningar
print("🚀 Kör benchmark...")

# Använd modellen från föregående notebook (eller skapa en snabb)
!python -m piedge_edukit.train --fakedata --no-pretrained --epochs 1 --batch-size 256 --output-dir ./models_bench


In [ ]:
# Benchmark med olika antal runs för att se varians
import os

# Test 1: Få runs (snabb)
print("📊 Test 1: 10 runs")
!python -m piedge_edukit.benchmark --fakedata --model-path ./models_bench/model.onnx --warmup 3 --runs 10 --providers CPUExecutionProvider


In [ ]:
# Visa benchmark-resultat
if os.path.exists("./reports/latency_summary.txt"):
    with open("./reports/latency_summary.txt", "r") as f:
        print("📈 Benchmark-resultat:")
        print(f.read())
else:
    print("❌ Benchmark-rapport saknas")


In [ ]:
# Läs detaljerade latensdata och visualisera
import pandas as pd
import matplotlib.pyplot as plt

if os.path.exists("./reports/latency.csv"):
    df = pd.read_csv("./reports/latency.csv")
    
    print(f"📊 Latens-statistik:")
    print(f"Antal mätningar: {len(df)}")
    print(f"Mean: {df['latency_ms'].mean():.2f} ms")
    print(f"Std: {df['latency_ms'].std():.2f} ms")
    print(f"Min: {df['latency_ms'].min():.2f} ms")
    print(f"Max: {df['latency_ms'].max():.2f} ms")
    
    # Histogram
    plt.figure(figsize=(10, 6))
    plt.hist(df['latency_ms'], bins=20, alpha=0.7, edgecolor='black')
    plt.xlabel('Latens (ms)')
    plt.ylabel('Antal')
    plt.title('Latens-distribution')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Box plot
    plt.figure(figsize=(8, 6))
    plt.boxplot(df['latency_ms'])
    plt.ylabel('Latens (ms)')
    plt.title('Latens Box Plot')
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("❌ Latens CSV saknas")


## 🤔 Reflektionsfrågor

<details>
<summary>💭 Varför är p95 viktigare än mean för edge deployment?</summary>

**Svar**: p95 (95:e percentilen) visar den värsta latensen som 95% av användarna upplever. Det är viktigare än mean eftersom:

- **Användarupplevelse**: En användare som får 100ms latens kommer att märka det, även om mean är 10ms
- **SLA-krav**: Många system har SLA-krav på p95 latens
- **Outliers**: Mean kan påverkas av extrema värden, p95 är mer robust

</details>

<details>
<summary>💭 Vad händer med latens-variansen när du ökar antal runs?</summary>

**Svar**: Med fler runs får vi:
- **Mer stabil statistik** - p50/p95 blir mer tillförlitliga
- **Bättre förståelse av varians** - ser om modellen är konsistent
- **Mindre påverkan av outliers** - enstaka långsamma körningar påverkar mindre

**Experiment**: Kör benchmark med 10, 50, 100 runs och jämför standardavvikelsen.

</details>


## 🎯 Ditt eget experiment

**Uppgift**: Kör benchmark med olika inställningar och jämför resultaten.

**Förslag**:
- Testa olika antal runs (10, 50, 100)
- Jämför warmup-effekten (0, 3, 10 warmup)
- Analysera variansen mellan körningar

**Kod att modifiera**:
```python
# Ändra dessa värden:
WARMUP_RUNS = 5
BENCHMARK_RUNS = 50

!python -m piedge_edukit.benchmark --fakedata --model-path ./models_bench/model.onnx --warmup {WARMUP_RUNS} --runs {BENCHMARK_RUNS} --providers CPUExecutionProvider
```


In [ ]:
# TODO: Implementera ditt experiment här
# Ändra värdena nedan och kör benchmark

WARMUP_RUNS = 5
BENCHMARK_RUNS = 50

print(f"🧪 Mitt experiment: warmup={WARMUP_RUNS}, runs={BENCHMARK_RUNS}")

# TODO: Kör benchmark med dina inställningar
# !python -m piedge_edukit.benchmark --fakedata --model-path ./models_bench/model.onnx --warmup {WARMUP_RUNS} --runs {BENCHMARK_RUNS} --providers CPUExecutionProvider


## 🎉 Sammanfattning

Du har nu lärt dig:
- Vad latens är och varför det är kritiskt för edge deployment
- Hur benchmark fungerar (warmup, runs, statistik)
- Hur man tolkar latens-resultat (p50, p95, varians)
- Varför p95 är viktigare än mean för användarupplevelse

**Nästa steg**: Gå till `03_quantization.ipynb` för att förstå hur kvantisering kan förbättra prestanda.

**Viktiga begrepp**:
- **Latens**: Inference-tid (kritiskt för edge)
- **Warmup**: Förbereder modellen för mätning
- **p50/p95**: Percentiler för latens-distribution
- **Varians**: Konsistens i prestanda
